# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import random
import torch
import numpy as np
import time
import pickle
from itertools import count
from collections import deque
import os
import matplotlib.pyplot as plt
%matplotlib inline

from buffer import ReplayBuffer
from maddpg import MADDPG
from utilities import transpose_list, transpose_to_tensor
#  conda install -c conda-forge tensorboardx  -- cause troubles with protobuf version
# rather pip install tensorboardX
from tensorboardX import SummaryWriter

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="../Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents = 2
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action = 2
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space -- Each observes a state with length: 24
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24


As pointed by my classmate [@jg1141](https://github.com/jg1141), the **observation space** consists of **8 variables** corresponding to
- the `position [x, y]`
- and `velocity [vx, vy]`
- of the `ball`
- and `racket`.

Each agent receives its own, local observation. **Two continuous actions** are available, corresponding to

- `movement toward (or away from) the net`
- and `jumping`. 

For each agent, the **"state"** issued by the environment is actually the **concatenation of 3 successive observations**. The most recent being at the end.

- Hence, for each agent `len(state) = 24`

Let's define a function to print the actions and states for the two agents.

In [5]:
# !pip install tabulate
# for one single agent
from tabulate import tabulate
def print_state(single_state, actions_deque, agent_name):
    full_actions = []
    for i in range(3):
        full_actions = full_actions + [""]*2  # padding
        full_actions.append("{:.3f}".format(actions_deque[i][0]))
        full_actions.append("{:.3f}".format(actions_deque[i][1]))
        full_actions = full_actions + [""]*4  # padding
    state_vector_names = ['racket pos x', 'racket pos y', 'racket vel x', 'racket vel y',
                          'ball pos x', 'ball pos y', 'ball vel x', 'ball vel y']
    res = []
    for i, e in enumerate(state_vector_names):
        line = []
        for k in range(3):
            line.append(full_actions[i+k*8])
            line.append(single_state[i+k*8])        
        line[0] = e
        res.append(line)
    headers = [agent_name, "s_t-2", "a_t-2", "s_t-1", "a_t-1", "s_t"]
    print(tabulate(res, headers=headers, numalign="right", floatfmt=".3f"))

# print states for both agents
def print_both_states(full_state):
    state_vector_names = ['racket pos x', 'racket pos y', 'racket vel x', 'racket vel y',
                          'ball pos x', 'ball pos y', 'ball vel x', 'ball vel y']
    res = []
    for i, e in enumerate(state_vector_names):
        line = [[full_state[0][i+k*8], full_state[1][i+k*8]] for k in range(3)]
        line = [e for l in line for e in l]
        line.insert(0, e)
        res.append(line)
    headers = ["", "st-2_A1", "st-2_A2", "st-1_A1", "st-1_A2", "st_A1", "st_A2"]
    print(tabulate(res, headers=headers, numalign="right", floatfmt=".3f"))

In [6]:
print('The initial states for the both agents look like:')
# print_both_states(states)
print_state(states[0], [[-1, -1], [0, 0], [1, 1]], "Agent_0")

The initial states for the both agents look like:
Agent_0         s_t-2  a_t-2      s_t-1  a_t-1       s_t
------------  -------  -------  -------  -------  ------
racket pos x    0.000             0.000           -6.653
racket pos y    0.000             0.000           -1.500
racket vel x    0.000  0.000      0.000  1.000    -0.000
racket vel y    0.000  0.000      0.000  1.000     0.000
ball pos x      0.000             0.000            6.832
ball pos y      0.000             0.000            6.000
ball vel x      0.000             0.000           -0.000
ball vel y      0.000             0.000            0.000


In [7]:
# env.close()

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [8]:
random_actions = False

In [9]:
def show(n_runs, random_flag):
    actions_deque = deque(maxlen=3)                            # collect 3 latest actions
    actions_deque.append([[0, 0], [0, 0]])
    actions_deque.append([[0, 0], [0, 0]])
    actions_deque.append([[0, 0], [0, 0]])
    for i in range(1, n_runs):
        env_info = env.reset(train_mode=False)[brain_name]
        states = env_info.vector_observations
        scores = np.zeros(num_agents)
        print_both_states(states)
        print_state(states[0], [a[0] for a in actions_deque], "Agent_0")
        print_state(states[1], [a[1] for a in actions_deque], "Agent_1")
        while True:
            if random_flag:
                actions = np.random.randn(num_agents, action_size)
                actions = np.clip(actions, -1, 1)
            else:
                actions = maddpg.act(transpose_to_tensor(obs), noise=0)
                actions_array = torch.stack(actions).cpu().detach().numpy()
                actions_for_env = np.rollaxis(actions_array, 1)
                actions = actions_for_env[0]
            actions_deque.append(actions)

            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            scores += env_info.rewards                         # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            print("actions = {}".format(actions))
            print("actions_deque = {}".format(actions_deque))
            print("states = {}".format(states))
            print_both_states(states)
            print_state(states[0], [a[0] for a in actions_deque], "Agent_0")
            print_state(states[1], [a[1] for a in actions_deque], "Agent_1")
            if np.any(dones):                                  # exit loop if episode finished
                print('\nDONE')
                break
        print('Score (max over agents) from episode {}: {}\n'.format(i, np.max(scores)))

In [10]:
if random_actions:
    show(n_runs=6, random=True)

When finished, you can close the environment.

In [11]:
if random_actions:
    env.close()

### 4. Training - Parameters

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [12]:
def seeding(seed=1):
    np.random.seed(seed)
    torch.manual_seed(seed)

In [13]:
# [x]o.p. refers to the values from original paper
# configuration and hyper-parameters.
seed_id = 1               
seeding(seed_id)          # [train with 10 random seeds]o.p.
n_agents = 2              # fix by the env
parallel_envs = 1         # cannot do anything but one

number_of_episodes = 10000
episode_lim = 1000        # number of steps to stop an episode 
batchsize = 128           # [1024]o.p. [64]ddpg.p.
# [after every 100 samples added to the replay buffer]o.p. means 
update_every = 2          # update period w.r.t. time steps
buffer_size = int(3e4)    # how many episodes kept as worth of replay
# DDPG and MADDPG speaks in terms of transitions: [10e6]o.p. and [10e6]ddpg.p.
discount_factor = 0.99    # [0.95]o.p.  [0.99]ddpg.p.
tau = 0.01                # [0.01]o.p.  [0.001]ddpg.p.
save_interval = 50        # how many episodes to save policy

# amplitude of OU noise -- slowly decrease noise to 0
noise = 1.0               #  2.0
mini_noise = 0.01
noise_reduction = 1.0     #  0.9999

In [14]:
# tensorboard logging
log_path = os.getcwd()+"/log"
model_dir= os.getcwd()+"/model_dir"
os.makedirs(model_dir, exist_ok=True)
logger = SummaryWriter(log_dir=log_path)
agent0_reward = []
agent1_reward = []

# in MADDPG: two agents but one shared buffer
maddpg = MADDPG(discount_factor=discount_factor, tau=tau)
buffer = ReplayBuffer(int(buffer_size))

# performance measurement
threshold_success = 0.5
scores_global = []                # collect scores for each episode
scores_deque = deque(maxlen=100)  # collect 100 latest scores
start_learning = time.time()
stop = False

# collect 3 latest actions
actions_deque = deque(maxlen=3)
actions_deque.append([[0, 0], [0, 0]])
actions_deque.append([[0, 0], [0, 0]])
actions_deque.append([[0, 0], [0, 0]])

# show progressbar
import progressbar as pb
widget = ['episode: ', pb.Counter(),'/',str(number_of_episodes),' ', 
          pb.Percentage(), ' ', pb.ETA(), ' ', pb.Bar(marker=pb.RotatingMarker()), ' ' ]
timer = pb.ProgressBar(widgets=widget, maxval=number_of_episodes).start()

### 5. Training - Main Loop
I decide to define `obs_full` as the concatenation of `(st-2, st-1, st)` for both agent.

Hence `np.shape(full_obs) = (1, 8*3*2) = (1, 48)`

On the other hand, `np.shape(full_obs) = (1, 2, 24)` is used to choose actions.

Instructions:

- After each episode, we add up the rewards that each agent received (without discounting) to get a score for each agent.

- This yields 2 (potentially different) scores.

- We then take the **maximum of these 2 scores**.

- This yields a **single score** for **each episode**.

- The environment is considered solved, when the **average (over 100 episodes)** of those scores is at least **+0.5**

In [15]:
for episode in range(0, number_of_episodes, parallel_envs):
    
    # book-keeping
    timer.update(episode)
    start_episode = time.time()
    reward_this_episode = np.zeros((parallel_envs, n_agents))

    # reset the environment
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations
    obs = [states]
    obs_full = [np.concatenate(states, axis=None)]
    
    # reset noise process
    maddpg.reset()

#     print_both_states(states)
#     print_state(states[0], [a[0] for a in actions_deque], "Agent_0")
#     print_state(states[1], [a[1] for a in actions_deque], "Agent_1")

    # save info and models or not
    save_info = ((episode) % save_interval < parallel_envs or episode==number_of_episodes-parallel_envs)

    for step_id in range(episode_lim):
        actions = maddpg.act(transpose_to_tensor(obs), noise=max(noise, mini_noise))
        noise *= noise_reduction
#         actions_array = torch.stack(actions).detach().numpy()
        actions_array = torch.stack(actions).cpu().detach().numpy()
        actions_for_env = np.rollaxis(actions_array, 1)
        actions = actions_for_env[0]
#         actions_deque.append(actions)

        # step forward one frame
        # next_obs, next_obs_full, rewards, dones, info = env.step(actions_for_env)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations 
        rewards = env_info.rewards
        dones = env_info.local_done

#         print_both_states(next_states)
#         print_state(next_states[0], [a[0] for a in actions_deque], "Agent_0")
#         print_state(next_states[1], [a[1] for a in actions_deque], "Agent_1")

        # adapt format
        next_obs_full = [np.concatenate(next_states, axis=None)]
        next_obs = [next_states]
        dones = [dones]
        rewards = [rewards]  # [[0.0, -0.01]]

        # add data to buffer
        transition = (obs, obs_full, actions_for_env, rewards, next_obs, next_obs_full, dones)
        buffer.push(transition)

        # before transitioning to new step
        reward_this_episode += rewards  # [[-0.16, -0.25]] -- shape = (1, 2)
        obs = next_obs

        # update models once after every "x" time steps
        if len(buffer) > batchsize and step_id % update_every < parallel_envs:
#             print("step_id = {} -- time for update with batchsize = {}".format(step_id, batchsize))
            for a_i in range(n_agents):
                # each agent does its own sampling from the shared replay buffer to do an update
                samples = buffer.sample(batchsize)  # shape = (7, Batch_Size)
                # i.e. for obs (samples[0]), np.shape = (Batch_Size, 2, 24)
                # and for obs_full (samples[1]), np.shape = (Batch_Size, 48)
                maddpg.update(samples, a_i, logger)
            maddpg.update_targets() #  soft update the target network towards the actual networks

    # --- the episode is over ---

    # measure performance
    score = max(reward_this_episode[0])
    scores_deque.append(score)
    scores_global.append(score)
    print('{}-th episode, score_ep= {:.4f}, avg_score= {:.4f}, duration[ep]= {:.2f} [s]'\
          .format(episode, score, np.mean(scores_deque), time.time()-start_episode), end="\n")
    for i in range(parallel_envs):
        agent0_reward.append(reward_this_episode[i,0])
        agent1_reward.append(reward_this_episode[i,1])
    
    # log performance
    logger.add_scalar('score', score, episode)
    logger.add_scalar('avg_score', np.mean(scores_deque), episode)
    if episode % 3 == 0 or episode == number_of_episodes-1:
        avg_rewards = [np.mean(agent0_reward), np.mean(agent1_reward)]
        agent0_reward = []
        agent1_reward = []
        for a_i, avg_rew in enumerate(avg_rewards):
            logger.add_scalar('agent%i/mean_episode_rewards' % a_i, avg_rew, episode)

    # Test if success
    if np.mean(scores_deque) > threshold_success:
        print("SUCCESS -- after {} episodes and {:.2f} min".format(episode, (time.time()-start_learning)/60))
        save_info = True
        stop = True

    # saving model
    save_dict_list =[]
    if save_info:
        saving_name = 'checkpoints/solved' if stop else 'checkpoints/episode_{}'.format(episode)
        maddpg.save(saving_name=saving_name)

    if stop:
        break

logger.close()
timer.finish()

0-th episode, score_ep= -0.0300, avg_score= -0.0300, duration[ep]= 6.64 [s]


1-th episode, score_ep= -0.0200, avg_score= -0.0250, duration[ep]= 9.41 [s]


2-th episode, score_ep= -0.0900, avg_score= -0.0467, duration[ep]= 9.40 [s]


3-th episode, score_ep= -0.0900, avg_score= -0.0575, duration[ep]= 9.58 [s]


4-th episode, score_ep= -0.0400, avg_score= -0.0540, duration[ep]= 9.85 [s]


...

25-th episode, score_ep= 0.0000, avg_score= -0.0096, duration[ep]= 8.63 [s]


26-th episode, score_ep= -0.0100, avg_score= -0.0096, duration[ep]= 8.39 [s]


27-th episode, score_ep= -0.0200, avg_score= -0.0100, duration[ep]= 8.79 [s]


28-th episode, score_ep= -0.0100, avg_score= -0.0100, duration[ep]= 9.17 [s]


29-th episode, score_ep= 0.0200, avg_score= -0.0090, duration[ep]= 8.79 [s]


In [16]:
# env.close()

### 6. Testing
Load and test a MADDPG agent

In [17]:
# Load the saved model weights of a successful agent
maddpg = MADDPG()

In [18]:
# save_dict = {'actor_params' : maddpg.maddpg_agent[i].local_actor.state_dict(),
#                          'actor_optim_params': maddpg.maddpg_agent[i].actor_optimizer.state_dict(),
#                          'critic_params' : maddpg.maddpg_agent[i].local_critic.state_dict(),
#                          'critic_optim_params' : maddpg.maddpg_agent[i].critic_optimizer.state_dict()}
#             save_dict_list.append(save_dict)
#             torch.save(save_dict_list, os.path.join(
#                 model_dir, 'episode-{}.pt'.format(episode)))


# state = torch.load()
maddpg.load("checkpoints/episode_0")

Loaded: checkpoints/episode_0.actor.pth
Loaded: checkpoints/episode_0.critic.pth


In [19]:
show(n_runs=12, random_flag=False)

                st-2_A1    st-2_A2    st-1_A1    st-1_A2    st_A1    st_A2
------------  ---------  ---------  ---------  ---------  -------  -------
racket pos x      0.000      0.000      0.000      0.000   -6.537   -6.433
racket pos y      0.000      0.000      0.000      0.000   -1.500   -1.500
racket vel x      0.000      0.000      0.000      0.000   -0.000    0.000
racket vel y      0.000      0.000      0.000      0.000    0.000    0.000
ball pos x        0.000      0.000      0.000      0.000    6.660   -6.660
ball pos y        0.000      0.000      0.000      0.000    1.022    1.022
ball vel x        0.000      0.000      0.000      0.000   -0.000    0.000
ball vel y        0.000      0.000      0.000      0.000    0.000    0.000
Agent_0         s_t-2  a_t-2      s_t-1  a_t-1       s_t
------------  -------  -------  -------  -------  ------
racket pos x    0.000             0.000           -6.537
racket pos y    0.000             0.000           -1.500
racket vel x    0.000 

 [-0.57951385 -0.54931825]]
actions_deque = deque([array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32)], maxlen=3)
states = [[  0.           0.           0.           0.           0.
    0.           0.           0.          -7.67573166  -1.5
   -0.           0.          -6.01283598   5.96076012  -0.
    0.          -4.83795929  -1.55886006  28.37771988  -0.98100001
   -6.01283598   5.82342005  28.37771988  -0.98100001]
 [  0.           0.           0.           0.           0.
    0.           0.           0.          -7.86340952  -1.5
    0.           0.           6.01283598   5.96076012   0.
    0.          -9.60195351  -1.55886006 -17.38541603  -0.98100001
    6.01283598   5.82342005 -17.38541603  -0.98100001]]
                st-2_A1    st-2_A2    st-1_A1    st-1_A2    st_A1  

actions = [[ 0.945924   -0.93011075]
 [-0.57951385 -0.54931825]]
actions_deque = deque([array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32)], maxlen=3)
states = [[-4.32445526e-01 -1.85216200e+00  2.83777199e+01  0.00000000e+00
  -6.01283598e+00  4.82280064e+00  2.83777199e+01  0.00000000e+00
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.01283598e+00  4.29306078e+00  2.83777199e+01 -4.76837158e-06
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.01283598e+00  3.66522074e+00  2.83777199e+01 -4.76837158e-06]
 [-1.08997889e+01 -1.85216177e+00 -9.53674316e-06  0.00000000e+00
   6.01283598e+00  4.82280064e+00 -9.53674316e-06  0.00000000e+00
  -1.08997889e+01 -1.85216177e+00 -9.53674316e-06  0.00000000e+00
   6.01283598e+00  4.29306078e+00 -9

actions = [[ 0.945924   -0.93011075]
 [-0.57951385 -0.54931825]]
actions_deque = deque([array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32)], maxlen=3)
states = [[-4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.01283598e+00  2.93928099e+00  2.83777199e+01 -4.76837158e-06
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.01283598e+00  2.11524105e+00  2.83777199e+01 -4.76837158e-06
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.01283598e+00  1.20638525e+00  2.83777199e+01 -4.76837158e-06]
 [-1.08997889e+01 -1.85216177e+00 -9.53674316e-06  0.00000000e+00
   6.01283598e+00  2.93928099e+00 -9.53674316e-06  0.00000000e+00
  -1.08997889e+01 -1.85216177e+00 -9.53674316e-06  0.00000000e+00
   6.01283598e+00  2.11524105e+00 -9

actions = [[ 0.945924   -0.93011075]
 [-0.57951385 -0.54931825]]
actions_deque = deque([array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32)], maxlen=3)
states = [[-4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.01283598e+00  2.86765218e-01  2.83777199e+01 -4.76837158e-06
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.01283598e+00 -6.32854819e-01  2.83777199e+01 -4.76837158e-06
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.01283598e+00 -1.55247474e+00  2.83777199e+01 -4.76837158e-06]
 [-1.08997889e+01 -1.85216177e+00 -9.53674316e-06  0.00000000e+00
   6.01283598e+00  2.86765218e-01 -9.53674316e-06  0.00000000e+00
  -1.08997889e+01 -1.85216177e+00 -9.53674316e-06  0.00000000e+00
   6.01283598e+00 -6.32854819e-01 -9

...

ball vel y      0.000             0.000             0.000
actions = [[ 0.945924   -0.93011075]
 [-0.57951385 -0.54931825]]
actions_deque = deque([array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32)], maxlen=3)
states = [[-4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -7.20544147e+00  2.62143707e+00  2.83777199e+01 -4.76837158e-06
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -7.20544147e+00  1.75815713e+00  2.83777199e+01 -4.76837158e-06
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -7.20544147e+00  8.38537276e-01  2.83777199e+01 -4.76837158e-06]
 [-1.08997889e+01 -1.85216248e+00 -9.53674316e-06  0.00000000e+00
   7.20544147e+00  2.62143707e+00 -9.53674316e-06  0.00000000e+00
  -1.08997889e+01 -1.85216248e+00 -9.5367431

actions = [[ 0.945924   -0.93011075]
 [-0.57951385 -0.54931825]]
actions_deque = deque([array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32)], maxlen=3)
states = [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.09210348 -1.5
  -0.          0.         -6.4263196   5.96076012 -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.00118589 -1.5
   0.          0.          6.4263196   5.96076012  0.          0.        ]]
                st-2_A1    st-2_A2    st-1_A1    st-1_A2    st_A1    st_A2
-------

actions = [[ 0.945924   -0.93011075]
 [-0.57951385 -0.54931825]]
actions_deque = deque([array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32)], maxlen=3)
states = [[-4.32445526e-01 -1.85235870e+00  2.83777199e+01  0.00000000e+00
  -6.42631960e+00  5.17596054e+00  2.83777199e+01  0.00000000e+00
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01  0.00000000e+00
  -6.42631960e+00  4.72470045e+00  2.83777199e+01  0.00000000e+00
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.42631960e+00  4.17534065e+00  2.83777199e+01 -4.76837158e-06]
 [-1.09001493e+01 -1.85235870e+00  0.00000000e+00  0.00000000e+00
   6.42631960e+00  5.17596054e+00  0.00000000e+00  0.00000000e+00
  -1.08997889e+01 -1.85216200e+00  0.00000000e+00  0.00000000e+00
   6.42631960e+00  4.72470045e+00  0

actions = [[ 0.945924   -0.93011075]
 [-0.57951385 -0.54931825]]
actions_deque = deque([array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32)], maxlen=3)
states = [[-4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.42631960e+00  1.93866110e+00  2.83777199e+01 -4.76837158e-06
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.42631960e+00  1.02246130e+00  2.83777199e+01 -4.76837158e-06
  -4.32445526e-01 -1.85216200e+00  2.83777199e+01 -4.76837158e-06
  -6.42631960e+00  1.02841213e-01  2.83777199e+01 -4.76837158e-06]
 [-1.08997889e+01 -1.85216200e+00 -9.53674316e-06  0.00000000e+00
   6.42631960e+00  1.93866110e+00 -9.53674316e-06  0.00000000e+00
  -1.08997889e+01 -1.85216200e+00 -9.53674316e-06  0.00000000e+00
   6.42631960e+00  1.02246130e+00 -9

 [-0.57951385 -0.54931825]]
actions_deque = deque([array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32), array([[ 0.945924  , -0.93011075],
       [-0.57951385, -0.54931825]], dtype=float32)], maxlen=3)
states = [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.4200449  -1.5
  -0.          0.         -6.55247116  5.97645617 -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.92566919 -1.5
   0.          0.          6.55247116  5.97645617  0.          0.        ]]
                st-2_A1    st-2_A2    st-1_A1    st-1_A2    st_A1    st_A2
------------  ---------  ---------  --------

In [20]:
env.close()